# External ID Lookup

Create a new table with one row per external_id, with the person_id that it corresponds to.

In [1]:
import os
os.chdir('..')

import pandas as pd

In [2]:
people = pd.read_csv('data/people.csv')

In [3]:
people[people.hash_ids.notnull()].hash_ids.str.split(', ')[225]

['94aa6bf43202356ced3ffe79b90da22607bd8a1d821ed4a296390eb0',
 'cca02dff0ab960320376773aba8e11d8071161aecbf4abd14b971ca6',
 '3720a25a371e99c6c93c78a4747e64fc6d6d281dfc9c2b0bab89f7db']

In [4]:
people['external_id_list'] = people.hash_ids.str.split(', ')

In [5]:
external_id_list = []

for idx, row in people[people.hash_ids.notnull()].iterrows():
    
    for eid in row.external_id_list:
        external_id_list += [[eid, row.person_id]]

In [6]:
# external_id_lookup = (
#     pd.DataFrame
#     .from_dict(external_id_dict, orient='index', columns=['person_id'])
#     .reset_index()
#     .rename(columns={'index': 'external_id'})
# )

In [7]:
external_id_lookup = (
    pd.DataFrame(external_id_list,columns=['external_id', 'person_id'])
#     .reset_index()
#     .rename(columns={'index': 'external_id'})
)
external_id_lookup

,external_id,person_id
0,94aa6bf43202356ced3ffe79b90da22607bd8a1d821ed4...,10227
1,cca02dff0ab960320376773aba8e11d8071161aecbf4ab...,10227
2,3720a25a371e99c6c93c78a4747e64fc6d6d281dfc9c2b...,10227


In [8]:
# external_id_lookup.to_csv('data/external_id_person_id_lookup.csv', index=False)

## Assemble existing external_id - person_id pairs

In [15]:
candidates = pd.read_csv('data/candidates.csv')

In [20]:
external_grouped_by_person = (
    candidates.loc[candidates.external_id.notnull(), ['external_id', 'person_id']]
    .groupby('person_id')
    .external_id.apply(', '.join)
    .reset_index()
)

In [21]:
people_to_gsheets = pd.merge(people, external_grouped_by_person, how='left', on='person_id')

In [24]:
people_to_gsheets[['person_id', 'external_id']].to_clipboard(index=False)